In [13]:
import pickle
import os
import re
import pandas as pd

In [14]:
file_pattern = '.*gssfold.*csv'
data_dir = '/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/feature_selection'
files = [f for f in os.listdir(data_dir) if re.match(file_pattern, f)]
files

['sel_feats_xtrees_gssfold1.csv',
 'sel_feats_xtrees_gssfold2.csv',
 'sel_feats_xtrees_gssfold5.csv',
 'sel_feats_xtrees_gssfold4.csv',
 'sel_feats_xtrees_gssfold3.csv']

In [16]:
df_list = []
i = 1
for f in files:
    df = pd.read_csv(data_dir+'/'+f)
    df['fold'] = int(i)
    i+=1
    df_list.append(df)

In [33]:
res = pd.concat(df_list)
res = res.sort_values(['feature','fold'], ascending=[True, True]).reset_index(drop=True)
res

,feature,mdi,fold
0,Mij,0.022206,1
1,Mij,0.011625,2
2,Mij,0.006565,3
3,Mij,0.005213,4
4,Mij,0.010536,5
...,...,...,...
524,zscore_bioplex2,0.015199,2
525,zscore_bioplex2,0.013859,3
526,zscore_bioplex2,0.017607,4
527,zscore_bioplex2,0.014773,5


In [36]:
gb = res.groupby(['feature'])
counts = gb.size().to_frame(name='counts')
agg_res = (counts
           .join(gb.agg({'fold': lambda x: ', '.join(set(x.astype(str).dropna()))}).rename(columns={'fold': 'gss_folds'}))
           .join(gb.agg({'mdi': 'mean'}).rename(columns={'mdi': 'mean_mdi'}))
           .join(gb.agg({'mdi': 'min'}).rename(columns={'mdi': 'min_mdi'}))
           .join(gb.agg({'mdi': 'max'}).rename(columns={'mdi': 'max_mdi'}))
           .sort_values(['counts', 'mean_mdi'], ascending=[False, False])
           .reset_index()
          )

In [37]:
agg_res

,feature,counts,gss_folds,mean_mdi,min_mdi,max_mdi
0,hein_neg_ln_pval,5,"3, 5, 2, 4, 1",0.048730,0.025006,0.075427
1,hein_pair_count,5,"3, 5, 2, 4, 1",0.042252,0.018668,0.051452
2,neg_ln_pval_bioplex2_Z4,5,"3, 5, 2, 4, 1",0.035888,0.024868,0.044570
3,neg_ln_pval,5,"3, 5, 2, 4, 1",0.028355,0.017168,0.038513
4,plate_zscore_bioplex2,5,"3, 5, 2, 4, 1",0.028175,0.021191,0.037386
...,...,...,...,...,...,...
226,tetts.sec_2.150p.braycurtis.feat,1,3,0.002837,0.002837,0.002837
227,wheat.sec_1.150p.pearsonR.feat,1,3,0.002758,0.002758,0.002758
228,caeel.iex_2.150p.pearsonR.feat,1,3,0.002737,0.002737,0.002737
229,xenla.sucrose_2.150p.spearmanR.feat,1,3,0.002708,0.002708,0.002708
